# Getting Started with MongoDB

Steps:
1. Install MongoDB here: https://docs.mongodb.com/manual/tutorial/install-mongodb-on-os-x/


2. Install pymongo here: https://api.mongodb.com/python/current/installation.html


3. Before using mongoDB, start mongodb at the command prompt.


4. Now you can use the pymongo package.

In [ ]:
from pymongo import MongoClient

client =  MongoClient('localhost',27017)

# Show existing databases
print(client.list_database_names())

## Getting Started with Mongo DB (courtesy of Thomas)

- review mongoDB tutorial
- discuss code (courtesy of Thomas)

In [37]:
import pymongo
from pymongo import MongoClient

peoplelist = [
    { "name": "Britney Sykes", "age": 21 , 'position':'Guard'}, 
    { "name": "Briana Day", "age": 19, 'position':'Center'}, 
    { "name": "Alexis Peterson", "age": 21, 'position':'Guard' }, 
    { "name": "Gabby Cooper", "age": 18, 'position':'Guard'}
]

# setup client
client = MongoClient('localhost', 27017)

# list db names
client.list_database_names()
# choose db
db = client.peopledb

# drop people collection if exists; start from scratch
if 'people' in db.list_collection_names():
    db.drop_collection('people')

# create new collection
peoplecoll = db.people


# insert peoplelist into peoplecoll
peoplecoll.insert_many(peoplelist)

# query entries with age less than 20
results = peoplecoll.find({'age':{'$lt':19}})
[print(result) for result in results]

{'_id': ObjectId('5dd4896a93e4423b240e99bc'), 'name': 'Gabby Cooper', 'age': 18, 'position': 'Guard'}


[None]

# JSON Example 

In [20]:
import urllib.request
import json

earthquake_url ="http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson"

response = urllib.request.urlopen(earthquake_url)

json_string = response.read().decode('utf-8')

eq_parsed_json = json.loads(json_string)

type(eq_parsed_json)

eq_parsed_json.keys() # Lists the keys at the top level

dict_keys(['type', 'metadata', 'features', 'bbox'])

In [25]:
eq_parsed_json['type']

eq_parsed_json['metadata']

title = eq_parsed_json['metadata']['title']

title

'USGS Significant Earthquakes, Past Month'

In [29]:
quakelist = eq_parsed_json['features']
quakelist

len(quakelist)

mags = []
# Create a list of the earthquake magnitudes
for quake in quakelist:
    mags.append(quake['properties']['mag'])

mags

[5.9, 7.1, 4.93, 6.5, 5.9, 3.42, 6.3, 6.6, 6.1, 6.5, 6.6, 6.3, 6.4]

In [30]:
# Printing the place of each earthquake
quakeplaces = [] # list of quake locations

for quake in quakelist:
    print(quake['properties']['place'])
    quakeplace = quake['properties']['place']
    quakeplaces.append(quakeplace)
    
quakeplaces[0:2]

4km WSW of South Hutchinson, Kansas
16km SE of Su'ao, Taiwan
54km ENE of Namie, Japan
102km WSW of Tugu Hilir, Indonesia
94km SW of San Antonio, Chile
62km E of Lakatoro, Vanuatu
24km S of La Libertad, El Salvador
140km ESE of Shingu, Japan
16km S of Twentynine Palms, CA
3km NNW of Magoula, Greece


['4km WSW of South Hutchinson, Kansas', "16km SE of Su'ao, Taiwan"]

## MongoDB with JSON (courtesy of Rebecca)

In [40]:
''' 
This program accesses the USGS earthquake data feed. This URL is for
       significant quakes in the last 30 days.
It then saves the earthquakes to a database named usgs and a collection called earthquakes.
The mongod server must be running for this program to work!
'''

import urllib.request
import json
import pymongo


# get the bbc rss feed of news stories and connect to it
earthquake_url = "http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson"

try:
    response = urllib.request.urlopen(earthquake_url)
except urllib.error.URLError as e:
    if hasattr(e, 'reason'):
        print('We failed to reach a server.')
        print('Reason: ', e.reason)
    elif hasattr(e, 'code'):
        print('The server couldn\'t fulfill the request.')
        print('Error code: ', e.code)
else:
    # the url request was successful - convert the response to a string
    json_string = response.read().decode('utf-8')

    # the json package loads() converts the string to python dictionaries and lists
    eq_json = json.loads(json_string)
    
    # from the json dictionary we get the title to print
    title = eq_json['metadata']['title']
    print('Collected data from', title)
    #  and we get the list of earthquakes
    quakelist = eq_json['features']

    # Connection to Mongo DB
    try:
        client=pymongo.MongoClient('localhost', 27017)
        print ("Connected successfully!!!")
    except pymongo.errors.ConnectionFailure as e:
        print ("Could not connect to MongoDB: %s" % e )
    else:

        # use database named usgs or create it if not there already
        eqdb = client.usgs
        # create collection named earthquakes or create it if not there already
        quakecoll = eqdb.earthquakes
        # add all the earthquakes to the list
        quakecoll.insert_many(quakelist)
        print("Added", len(quakelist), "to earthquakes collection in usgs database")
  
# for item in quakecoll.find({},{"mag":{'$lt':5}}):
 #   print(item)
  
# close the database connection
client.close()

Collected data from USGS Significant Earthquakes, Past Month
Connected successfully!!!
Added 13 to earthquakes collection in usgs database


OperationFailure: Unsupported projection option: mag: { $lt: 5 }

# Breakout Group Exercise

In [31]:
# Create a list of each of coordinates from "quakelist"
# ie  'coordinates': [-97.9879, 38.0101, 5]},

In [5]:
quake1 = quakelist[0]
type(quake1)
quake1.keys()

dict_keys(['type', 'properties', 'geometry', 'id'])

In [6]:
print(json.dumps(quake1, indent=2))

{
  "type": "Feature",
  "properties": {
    "mag": 4.2,
    "place": "4km WSW of South Hutchinson, Kansas",
    "time": 1565960350388,
    "updated": 1566045597598,
    "tz": -360,
    "url": "https://earthquake.usgs.gov/earthquakes/eventpage/us600055xa",
    "detail": "https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/us600055xa.geojson",
    "felt": 1019,
    "cdi": 6.1,
    "mmi": 5.593,
    "alert": null,
    "status": "reviewed",
    "tsunami": 0,
    "sig": 881,
    "net": "us",
    "code": "600055xa",
    "ids": ",us600055xa,",
    "sources": ",us,",
    "types": ",dyfi,geoserve,moment-tensor,origin,phase-data,shakemap,",
    "nst": null,
    "dmin": 0.702,
    "rms": 0.29,
    "gap": 186,
    "magType": "mwr",
    "type": "earthquake",
    "title": "M 4.2 - 4km WSW of South Hutchinson, Kansas"
  },
  "geometry": {
    "type": "Point",
    "coordinates": [
      -97.9879,
      38.0101,
      5
    ]
  },
  "id": "us600055xa"
}
